First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [3]:
import chardet
with open(f'{path}/data/elections/btw/es/btw21_es.csv', 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [4]:
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

btw02_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW02_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW05_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW09_Zweitstimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW13_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_zs = pd.read_csv(f'{path}/data/elections/btw/zs/btw17_wbz_zweitstimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/1795473645.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/1795473645.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/1795473645.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_c

View the data

In [5]:
btw17_es.head()

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,Bündnis C,DIE EINHEIT,DIE VIOLETTEN,FAMILIE,DIE FRAUEN,MIETERPARTEI,Neue Liberale,UNABHÄNGIGE,WGr/EB,Ungekürzte Wahlbezirksbezeichnung
0,1,01,0,01,0,000,00,000001,0,1670,...,0,0,0,0,0,0,0,0,7,–
1,1,01,0,01,0,000,00,000002,0,1671,...,0,0,0,0,0,0,0,0,6,–
2,1,01,0,01,0,000,00,000003,0,1603,...,0,0,0,0,0,0,0,0,1,–
3,1,01,0,01,0,000,00,000004,0,1497,...,0,0,0,0,0,0,0,0,5,–
4,1,01,0,01,0,000,00,000005,0,1251,...,0,0,0,0,0,0,0,0,4,–


Add year column to dataframes

In [6]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
years = [2002, 2005, 2009, 2013, 2017, 2002, 2005, 2009, 2013, 2017]
for df, year in zip(dfs, years):
    df.insert(loc=0, column='year', value=year)

In [7]:
btw17_zs.head()

,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,...,DKP,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³,Ungekürzte Wahlbezirksbezeichnung
0,2017,1,01,0,01,0,000,00,000001,0,...,0,0,0,0,0,0,0,0,0,–
1,2017,1,01,0,01,0,000,00,000002,0,...,0,0,0,0,0,0,0,0,0,–
2,2017,1,01,0,01,0,000,00,000003,0,...,0,0,0,0,0,0,0,0,0,–
3,2017,1,01,0,01,0,000,00,000004,0,...,0,0,0,0,0,0,0,0,0,–
4,2017,1,01,0,01,0,000,00,000005,0,...,0,0,0,0,0,0,0,0,0,–


Since some municipalities share a ballot vote election board, in this case, postal ballots cannot be attributed to a single municipality. 
In the 2002 to 2017 data, these cases were signified with '999' as Gemeindekennziffer. From 2017 on the Gemeindekennziffer is given by 9 + Kennziffer Briefwahlzugehörigkeit.

To drop the 999 Gemeinde in for 2002-2013, first append these dataframes

In [8]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es]
btw0213_es = pd.concat(dfs, sort=False)
dfs = [btw02_zs, btw05_zs, btw09_zs, btw13_zs]
btw0213_zs = pd.concat(dfs, sort=False)
btw0213_zs.head()

,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,ÖDP,Bündnis 21/RRP,AfD,BIG,pro Deutschland,DIE RECHTE,FREIE WÄHLER,Partei der Nichtwähler,PARTEI DER VERNUNFT,Die PARTEI
0,2002,1,01,0,01,0,000,1,0,1463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,1,01,0,01,0,000,2,0,1245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,1,01,0,01,0,000,3,0,1399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,1,01,0,01,0,000,4,0,1031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,1,01,0,01,0,000,5,0,1023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Then drop all observations with Gemeinde == 999

In [9]:
btw0213_es = btw0213_es[btw0213_es['Gemeinde'] != '999']
btw0213_zs = btw0213_zs[btw0213_zs['Gemeinde'] != '999']

For 2017, drop all observations where Gemeinde == 9 + Kennziffer Briefwahlzugehörigkeit

In [10]:
btw17_es = btw17_es[btw17_es['Gemeinde'] != '9' + btw17_es['Kennziffer Briefwahlzugehörigkeit']]
btw17_zs = btw17_zs[btw17_zs['Gemeinde'] != '9' + btw17_zs['Kennziffer Briefwahlzugehörigkeit']]

Now contatenate the dfs for 2002-13 with 2017

In [11]:
dfs = [btw0213_es, btw17_es]
btw_es = pd.concat(dfs, sort=False)
dfs = [btw0213_zs, btw17_zs]
btw_zs = pd.concat(dfs, sort=False)

Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [12]:
btw_es.insert(loc=0, column='AGS', value=0)
btw_es['AGS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Gemeinde']
btw_zs.insert(loc=0, column='AGS', value=0)
btw_zs['AGS'] = btw_zs['Land'] + btw_zs['Regierungsbezirk'] + btw_zs['Kreis'] + btw_zs['Gemeinde']
btw_zs.head()

,AGS,year,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Wahlbezirk,Bezirksart,...,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³,Ungekürzte Wahlbezirksbezeichnung
0,01001000,2002,1,01,0,01,0,000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,2002,1,01,0,01,0,000,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001000,2002,1,01,0,01,0,000,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001000,2002,1,01,0,01,0,000,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01001000,2002,1,01,0,01,0,000,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since there is only one polygon with the AGS 11000000 for Berlin, replace AGS values here

In [13]:
btw_es['AGS'] = np.where(btw_es['Land'] == '11', '11000000', btw_es['AGS']) #if condition, = 110..., else AGS
btw_zs['AGS'] = np.where(btw_zs['Land'] == '11', '11000000', btw_es['AGS'])

Collapse data along AGS column, sums votes over all constituencies in a municipality

In [14]:
btw_es = btw_es.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs = btw_zs.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs.head()

,AGS,year,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),Wahlberechtigte nach § 25 Abs. 2 BWO (A3),...,BGE,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³
0,01001000,2002,01,53,0,35,65740,57788,7949,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,01,53,0,35,66970,58379,8591,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,01,58,1500,55,60283,53341,6942,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,01,56,0,60,70321,60755,9566,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,01,56,0,60,70105,57929,12176,0.0,...,422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add indicator for first and second vote

In [15]:
dfs = [btw_es, btw_zs]
eszs = ['first_vote', 'second_vote']
for df, stimme in zip(dfs, eszs):
    df.insert(loc=2, column=stimme, value=1)
btw_zs.head()

,AGS,year,second_vote,Land,Wahlkreis,Verbandsgemeinde,Bezirksart,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),...,BGE,DiB,DM,Die Grauen,du.,MG,MENSCHLICHE WELT,Die Humanisten,Gesundheitsforschung,V-Partei³
0,01001000,2002,1,01,53,0,35,65740,57788,7949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01001000,2005,1,01,53,0,35,66970,58379,8591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01001000,2009,1,01,58,1500,55,60283,53341,6942,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01001000,2013,1,01,56,0,60,70321,60755,9566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01001000,2017,1,01,56,0,60,70105,57929,12176,...,422.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ES: Sum votes for other, Union and Die Linke parties

In [16]:
# list(btw_es.columns)

In [17]:
andere = [ 'REP',
 'GRAUE',
 'TIERSCHUTZ',
 'NPD',
 'ÖDP',
 'PBC',
 'FRAUEN',
 'BP',
 'FAMILIE',
 'CM',
 'BÜSO',
 'HP',
 'VIOLETTEN',
 'AUFBRUCH',
 'ZENTRUM',
 'KPD',
 'PRG',
 'SCHILL',
 'DEUTSCHLAND',
 'DKP',
 'DSU',
 'FP D',
 'ÜBRIGE',
 'Offensive D',
 'Die Tierschutzpartei',
 'DIE FRAUEN',
 'BüSo',
 'Deutschland',
 'AGFG',
 'APPD',
 'MLPD',
 'DIE PARTEI',
 'HUMANWIRTSCHAFTSPARTEI',
 'STATT Partei',
 'UNABHÄNGIGE',
 'Übrige',
 'Volksabstimmung',
 'ADM',
 'DIE VIOLETTEN',
 'ödp',
 'PIRATEN',
 'RRP',
 'Freie Union',
 'Tierschutzpartei',
 'Bündnis 21/RRP',
 'RENTNER',
 'AfD',
 'BIG',
 'pro Deutschland',
 'FREIE WÄHLER',
 'PARTEI DER VERNUNFT',
 'Die PARTEI',
 'Bergpartei',
 'BGD',
 'NEIN',
 'WGr/EB',
 'PDV',
 'SGP',
 'DIE RECHTE',
 'Tierschutzallianz',
 'B*',
 'Die Grauen',
 'du',
 'MG',
 'MENSCHLICHE WELT',
 'Gesundheitsforschung',
 'V-Partei³',
 'Bündnis C',
 'DIE EINHEIT',
 'MIETERPARTEI',
 'Neue Liberale']
btw_es['Andere'] = btw_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_es['Union'] = btw_es[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_es['Linke'] = btw_es[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_es['Ungültig'] = btw_es[invalid].sum(axis=1) # sum invalid votes
valid = ['Gültige', 'Gültig']
btw_es['Gültig'] = btw_es[valid].sum(axis=1) # sum valid votes

Drop not needed columns

In [18]:
columns_to_drop = andere + union + linke + ['Ungültige', 'Gültige', 'Wahlkreis', 'Verbandsgemeinde', 'Bezirksart']
btw_es.drop(columns_to_drop, axis=1, inplace=True)
btw_es.head()

,AGS,year,first_vote,Land,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),Wahlberechtigte nach § 25 Abs. 2 BWO (A3),Wähler (B),Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,GRÜNE,FDP,Andere,Union,Linke
0,01001000,2002,1,01,65740,57788,7949,3.0,49054,7573,503.0,48551.0,26309,2833,2038,194.0,16418,759.0
1,01001000,2005,1,01,66970,58379,8591,0.0,49002,8218,758.0,48244.0,24261,2729,1601,387.0,16954,2312.0
2,01001000,2009,1,01,69075,60983,8092,0.0,46583,7238,1204.0,45379.0,16552,5520,4253,498.0,13859,4697.0
3,01001000,2013,1,01,70321,60755,9566,0.0,46827,9084,589.0,46238.0,18915,4533,807,3086.0,15647,3250.0
4,01001000,2017,1,01,70105,57929,12176,0.0,49694,11528,485.0,49209.0,15175,5958,3421,4199.0,15140,5316.0


In [19]:
btw_es.columns

Index(['AGS', 'year', 'first_vote', 'Land', 'Wahlberechtigte (A)',
       'Wahlberechtigte ohne Sperrvermerk (A1)',
       'Wahlberechtigte mit Sperrvermerk (A2)',
       'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)', 'Wähler (B)',
       'Wähler mit Wahlschein (B1)', 'Ungültig', 'Gültig', 'SPD', 'GRÜNE',
       'FDP', 'Andere', 'Union', 'Linke'],
      dtype='object')

ZS: Sum votes for other, Union and DIE LINKE parties

In [20]:
# list(btw_zs.columns)

In [21]:
andere = [ 'REP',
 'GRAUE',
 'Tierschutz',
 'NPD',
 'ödp',
 'PBC',
 'FRAUEN',
 'BP',
 'FAMILIE',
 'CM',
 'BüSo',
 'HP',
 'Violetten',
 'AUFBRUCH',
 'ZENTRUM',
 'KPD',
 'PRG',
 'Schill',
 'Offensive D',
 'Die Tierschutzpartei',
 'DIE FRAUEN',
 'Deutschland',
 'AGFG',
 'APPD',
 '50plus',
 'MLPD',
 'Die Partei',
 'PSG',
 'Pro DM',
 'Volksabstimmung',
 'ADM',
 'DKP',
 'DVU',
 'DIE VIOLETTEN',
 'FWD',
 'PIRATEN',
 'RRP',
 'RENTNER',
 'Tierschutzpartei',
 'ÖDP',
 'Bündnis 21/RRP',
 'AfD',
 'BIG',
 'pro Deutschland',
 'DIE RECHTE',
 'FREIE WÄHLER',
 'Partei der Nichtwähler',
 'PARTEI DER VERNUNFT',
 'Die PARTEI',
 'PDV',
 'SGP',
 'AD-DEMOKRATEN',
 'Tierschutzallianz',
 'B*',
 'BGE',
 'DiB',
 'DM',
 'Die Grauen',
 'du.',
 'MG',
 'MENSCHLICHE WELT',
 'Die Humanisten',
 'Gesundheitsforschung',
 'V-Partei³']
btw_zs['Andere'] = btw_zs[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_zs['Union'] = btw_zs[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_zs['Linke'] = btw_zs[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_zs['Ungültig'] = btw_zs[invalid].sum(axis=1) # sum invalid votes
valid = ['Gültige', 'Gültig']
btw_zs['Gültig'] = btw_zs[valid].sum(axis=1) # sum valid votes

In [22]:
columns_to_drop = andere + union + linke + ['Ungültige', 'Gültige', 'Wahlkreis', 'Verbandsgemeinde', 'Bezirksart']
btw_zs.drop(columns_to_drop, axis=1, inplace=True)
btw_zs.head()

,AGS,year,second_vote,Land,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),Wahlberechtigte nach § 25 Abs. 2 BWO (A3),Wähler (B),Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,GRÜNE,FDP,Andere,Union,Linke
0,01001000,2002,1,01,65740,57788,7949,3.0,49054,7573,501.0,48553.0,23396,5556,3424,956.0,14362,859.0
1,01001000,2005,1,01,66970,58379,8591,0.0,49002,8218,767.0,48235.0,21000,5028,4196,1035.0,13910,3066.0
2,01001000,2009,1,01,60283,53341,6942,0.0,40926,7285,1058.0,39868.0,11446,6204,5603,1958.0,10382,4275.0
3,01001000,2013,1,01,70321,60755,9566,0.0,46827,9084,562.0,46265.0,16480,5507,2061,4301.0,14136,3780.0
4,01001000,2017,1,01,70105,57929,12176,0.0,49694,11528,423.0,49271.0,12142,7416,5073,5559.0,13382,5699.0


Concatenate first and second vote datasets

In [23]:
dfs = [btw_es, btw_zs]
btw_eszs = pd.concat(dfs, sort=False)
btw_eszs = btw_eszs.sort_values(['AGS', 'year'])
btw_eszs.head(12)

,AGS,year,first_vote,Land,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),Wahlberechtigte mit Sperrvermerk (A2),Wahlberechtigte nach § 25 Abs. 2 BWO (A3),Wähler (B),Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,GRÜNE,FDP,Andere,Union,Linke,second_vote
0,01001000,2002,1.0,01,65740,57788,7949,3.0,49054,7573,503.0,48551.0,26309,2833,2038,194.0,16418,759.0,NaN
0,01001000,2002,NaN,01,65740,57788,7949,3.0,49054,7573,501.0,48553.0,23396,5556,3424,956.0,14362,859.0,1.0
1,01001000,2005,1.0,01,66970,58379,8591,0.0,49002,8218,758.0,48244.0,24261,2729,1601,387.0,16954,2312.0,NaN
1,01001000,2005,NaN,01,66970,58379,8591,0.0,49002,8218,767.0,48235.0,21000,5028,4196,1035.0,13910,3066.0,1.0
2,01001000,2009,1.0,01,69075,60983,8092,0.0,46583,7238,1204.0,45379.0,16552,5520,4253,498.0,13859,4697.0,NaN
2,01001000,2009,NaN,01,60283,53341,6942,0.0,40926,7285,1058.0,39868.0,11446,6204,5603,1958.0,10382,4275.0,1.0
3,01001000,2013,1.0,01,70321,60755,9566,0.0,46827,9084,589.0,46238.0,18915,4533,807,3086.0,15647,3250.0,NaN
3,01001000,2013,NaN,01,70321,60755,9566,0.0,46827,9084,562.0,46265.0,16480,5507,2061,4301.0,14136,3780.0,1.0
4,01001000,2017,1.0,01,70105,57929,12176,0.0,49694,11528,485.0,49209.0,15175,5958,3421,4199.0,15140,5316.0,NaN
4,01001000,2017,NaN,01,70105,57929,12176,0.0,49694,11528,423.0,49271.0,12142,7416,5073,5559.0,13382,5699.0,1.0


Replace Land values with corresponding state abbreviation

In [24]:
states = {'01': 'SH', '02': 'HH', '03': 'NI', '04': 'HB', '05': 'NW', '06': 'HE', '07': 'RP', '08': 'BW', \
         '09': 'BY', '10': 'SL', '11': 'BE', '12': 'BB', '13': 'MV', '14': 'SN', '15': 'ST', '16': 'TH'}
for key, value in states.items():
    btw_eszs['Land'] = np.where(btw_eszs['Land'] == key, value, btw_eszs['Land'])
btw_eszs.Land.unique()

array(['SH', 'HH', 'NI', 'HB', 'NW', 'HE', 'RP', 'BW', 'BY', 'SL', 'BE',
       'BB', 'MV', 'SN', 'ST', 'TH'], dtype=object)

Add post-treatment indicator

In [25]:
btw_posttreat = btw_eszs #defragment df
years = [2017, 2013, 2009, 2005]
for year in years:
    btw_posttreat.insert(loc=3, column=f'post_{year}', value=0)
    btw_posttreat[f'post_{year}'] = np.where(btw_posttreat['year'] >= year, 1, 0)
btw_posttreat.head(10)

,AGS,year,first_vote,post_2005,post_2009,post_2013,post_2017,Land,Wahlberechtigte (A),Wahlberechtigte ohne Sperrvermerk (A1),...,Wähler mit Wahlschein (B1),Ungültig,Gültig,SPD,GRÜNE,FDP,Andere,Union,Linke,second_vote
0,01001000,2002,1.0,0,0,0,0,SH,65740,57788,...,7573,503.0,48551.0,26309,2833,2038,194.0,16418,759.0,NaN
0,01001000,2002,NaN,0,0,0,0,SH,65740,57788,...,7573,501.0,48553.0,23396,5556,3424,956.0,14362,859.0,1.0
1,01001000,2005,1.0,1,0,0,0,SH,66970,58379,...,8218,758.0,48244.0,24261,2729,1601,387.0,16954,2312.0,NaN
1,01001000,2005,NaN,1,0,0,0,SH,66970,58379,...,8218,767.0,48235.0,21000,5028,4196,1035.0,13910,3066.0,1.0
2,01001000,2009,1.0,1,1,0,0,SH,69075,60983,...,7238,1204.0,45379.0,16552,5520,4253,498.0,13859,4697.0,NaN
2,01001000,2009,NaN,1,1,0,0,SH,60283,53341,...,7285,1058.0,39868.0,11446,6204,5603,1958.0,10382,4275.0,1.0
3,01001000,2013,1.0,1,1,1,0,SH,70321,60755,...,9084,589.0,46238.0,18915,4533,807,3086.0,15647,3250.0,NaN
3,01001000,2013,NaN,1,1,1,0,SH,70321,60755,...,9084,562.0,46265.0,16480,5507,2061,4301.0,14136,3780.0,1.0
4,01001000,2017,1.0,1,1,1,1,SH,70105,57929,...,11528,485.0,49209.0,15175,5958,3421,4199.0,15140,5316.0,NaN
4,01001000,2017,NaN,1,1,1,1,SH,70105,57929,...,11528,423.0,49271.0,12142,7416,5073,5559.0,13382,5699.0,1.0


Drop not needed columns and rename

In [26]:
btw_posttreat.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'GRÜNE': 'Grüne', 'Wähler (B)': 'Wähler'}, inplace = True)
btw_posttreat = btw_posttreat[['AGS', 'Land', 'year', 'post_2005', 'post_2009', 'post_2013', 'post_2017', 'first_vote', 'second_vote', 'Wahlberechtigte', 'Wähler', 'Gültig', 'Ungültig', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw_posttreat.head(10)

,AGS,Land,year,post_2005,post_2009,post_2013,post_2017,first_vote,second_vote,Wahlberechtigte,Wähler,Gültig,Ungültig,Union,SPD,FDP,Linke,Grüne,Andere
0,01001000,SH,2002,0,0,0,0,1.0,NaN,65740,49054,48551.0,503.0,16418,26309,2038,759.0,2833,194.0
0,01001000,SH,2002,0,0,0,0,NaN,1.0,65740,49054,48553.0,501.0,14362,23396,3424,859.0,5556,956.0
1,01001000,SH,2005,1,0,0,0,1.0,NaN,66970,49002,48244.0,758.0,16954,24261,1601,2312.0,2729,387.0
1,01001000,SH,2005,1,0,0,0,NaN,1.0,66970,49002,48235.0,767.0,13910,21000,4196,3066.0,5028,1035.0
2,01001000,SH,2009,1,1,0,0,1.0,NaN,69075,46583,45379.0,1204.0,13859,16552,4253,4697.0,5520,498.0
2,01001000,SH,2009,1,1,0,0,NaN,1.0,60283,40926,39868.0,1058.0,10382,11446,5603,4275.0,6204,1958.0
3,01001000,SH,2013,1,1,1,0,1.0,NaN,70321,46827,46238.0,589.0,15647,18915,807,3250.0,4533,3086.0
3,01001000,SH,2013,1,1,1,0,NaN,1.0,70321,46827,46265.0,562.0,14136,16480,2061,3780.0,5507,4301.0
4,01001000,SH,2017,1,1,1,1,1.0,NaN,70105,49694,49209.0,485.0,15140,15175,3421,5316.0,5958,4199.0
4,01001000,SH,2017,1,1,1,1,NaN,1.0,70105,49694,49271.0,423.0,13382,12142,5073,5699.0,7416,5559.0


Since vote percentages are calculated based on the 'valid' votes and there are some rows with valid == 0, calculate the vote shares for these (Voters - Invalid)

In [27]:
btw_posttreat.loc[:, 'Gültig'] = np.where(btw_posttreat['Gültig'] == 0, btw_posttreat['Wähler'] - btw_posttreat['Ungültig'], btw_posttreat['Gültig'])

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/486520081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat.loc[:, 'Gültig'] = np.where(btw_posttreat['Gültig'] == 0, btw_posttreat['Wähler'] - btw_posttreat['Ungültig'], btw_posttreat['Gültig'])


Check summation of party and invalid votes

In [28]:
parties = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Ungültig']
btw_posttreat.loc[:, 'votes'] = btw_posttreat[parties].sum(axis=1)
btw_posttreat.loc[:, 'votes'] = btw_posttreat['votes'].astype(int) #convert to integer
print(btw_posttreat['votes'].equals(btw_posttreat['Wähler'])) # check if votes == Wähler
btw_posttreat.drop(['votes'], axis=1, inplace=True)

True


/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/3029531782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat.loc[:, 'votes'] = btw_posttreat[parties].sum(axis=1)
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/3029531782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat.loc[:, 'votes'] = btw_posttreat['votes'].astype(int) #convert to integer
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/3029531782.py:5: SettingWithCopyWarni

Check summation of valid and invalid votes

In [29]:
votes = ['Gültig', 'Ungültig']
btw_posttreat['votes'] = btw_posttreat[votes].sum(axis=1)
btw_posttreat['votes'] = btw_posttreat['votes'].astype(int) #convert to integer
print(btw_posttreat['votes'].equals(btw_posttreat['Wähler'])) # check if votes == Wähler
btw_posttreat.drop(['votes', 'Ungültig'], axis=1, inplace=True)

True


/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/2056866328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat['votes'] = btw_posttreat[votes].sum(axis=1)
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/2056866328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat['votes'] = btw_posttreat['votes'].astype(int) #convert to integer
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/2056866328.py:5: SettingWithCopyWarning: 
A value is 

Calculate vote shares in percent

In [30]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    btw_posttreat.loc[:, party] = (btw_posttreat[party] / btw_posttreat['Gültig'])*100
btw_posttreat.loc[:, 'Wähler'] = (btw_posttreat['Wähler'] / btw_posttreat['Wahlberechtigte'])*100
btw_posttreat.head(10)

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/2437638836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat.loc[:, party] = (btw_posttreat[party] / btw_posttreat['Gültig'])*100
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_88119/2437638836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_posttreat.loc[:, 'Wähler'] = (btw_posttreat['Wähler'] / btw_posttreat['Wahlberechtigte'])*100


,AGS,Land,year,post_2005,post_2009,post_2013,post_2017,first_vote,second_vote,Wahlberechtigte,Wähler,Gültig,Union,SPD,FDP,Linke,Grüne,Andere
0,01001000,SH,2002,0,0,0,0,1.0,NaN,65740,74.618193,48551.0,33.815987,54.188379,4.197648,1.563305,5.835101,0.399580
0,01001000,SH,2002,0,0,0,0,NaN,1.0,65740,74.618193,48553.0,29.580047,48.186518,7.052087,1.769201,11.443165,1.968982
1,01001000,SH,2005,1,0,0,0,1.0,NaN,66970,73.170076,48244.0,35.142194,50.288119,3.318547,4.792306,5.656662,0.802172
1,01001000,SH,2005,1,0,0,0,NaN,1.0,66970,73.170076,48235.0,28.837981,43.536851,8.699077,6.356380,10.423966,2.145745
2,01001000,SH,2009,1,1,0,0,1.0,NaN,69075,67.438292,45379.0,30.540558,36.475021,9.372177,10.350603,12.164217,1.097424
2,01001000,SH,2009,1,1,0,0,NaN,1.0,60283,67.889787,39868.0,26.040935,28.709742,14.053878,10.722886,15.561352,4.911207
3,01001000,SH,2013,1,1,1,0,1.0,NaN,70321,66.590350,46238.0,33.840131,40.907911,1.745318,7.028851,9.803625,6.674164
3,01001000,SH,2013,1,1,1,0,NaN,1.0,70321,66.590350,46265.0,30.554415,35.620880,4.454771,8.170323,11.903167,9.296444
4,01001000,SH,2017,1,1,1,1,1.0,NaN,70105,70.885101,49209.0,30.766730,30.837855,6.951980,10.802902,12.107541,8.532992
4,01001000,SH,2017,1,1,1,1,NaN,1.0,70105,70.885101,49271.0,27.159993,24.643299,10.296117,11.566642,15.051450,11.282499


Clean dataset

In [31]:
btw = btw_posttreat[['AGS', 'year', 'Land', 'post_2005', 'post_2009', 'post_2013', 'post_2017', 'first_vote', 'second_vote', 'Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw.head()

,AGS,year,Land,post_2005,post_2009,post_2013,post_2017,first_vote,second_vote,Wähler,Union,SPD,FDP,Linke,Grüne,Andere
0,01001000,2002,SH,0,0,0,0,1.0,NaN,74.618193,33.815987,54.188379,4.197648,1.563305,5.835101,0.399580
0,01001000,2002,SH,0,0,0,0,NaN,1.0,74.618193,29.580047,48.186518,7.052087,1.769201,11.443165,1.968982
1,01001000,2005,SH,1,0,0,0,1.0,NaN,73.170076,35.142194,50.288119,3.318547,4.792306,5.656662,0.802172
1,01001000,2005,SH,1,0,0,0,NaN,1.0,73.170076,28.837981,43.536851,8.699077,6.356380,10.423966,2.145745
2,01001000,2009,SH,1,1,0,0,1.0,NaN,67.438292,30.540558,36.475021,9.372177,10.350603,12.164217,1.097424


Find out the incumbent for each municipality/year --> largest share of first votes
First, create new df and drop na values for voting results

In [32]:
btw_incumbent = btw[btw['first_vote'] == 1]
btw_incumbent = btw_incumbent[['AGS', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw_incumbent = btw_incumbent.dropna(axis='rows', how='any', subset=['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'])
btw_incumbent.head()

,AGS,year,Union,SPD,FDP,Linke,Grüne,Andere
0,01001000,2002,33.815987,54.188379,4.197648,1.563305,5.835101,0.399580
1,01001000,2005,35.142194,50.288119,3.318547,4.792306,5.656662,0.802172
2,01001000,2009,30.540558,36.475021,9.372177,10.350603,12.164217,1.097424
3,01001000,2013,33.840131,40.907911,1.745318,7.028851,9.803625,6.674164
4,01001000,2017,30.766730,30.837855,6.951980,10.802902,12.107541,8.532992


Which party had the largest first vote share?

In [33]:
btw_incumbent['incumbent'] = btw_incumbent[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)
btw_incumbent['incumbent'].describe()

count     59850
unique        6
top       Union
freq      43831
Name: incumbent, dtype: object

Does the incumbent win?

In [34]:
btw_incumbent['inc_win'] = btw_incumbent['incumbent'].eq(btw_incumbent['incumbent'].shift())
btw_incumbent['inc_win'] = np.where(btw_incumbent['year'] == 2002, np.nan, btw_incumbent['inc_win'])
btw_incumbent['inc_win'].describe()

count    46604.000000
mean         0.843812
std          0.363038
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: inc_win, dtype: float64

Whats the incumbent vote share?

Merge with treatment dataset

In [35]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})
btw_treat = pd.merge(btw, treatment, on='AGS', how='outer', indicator=True)
btw_treat.head(10)

,AGS,year,Land,post_2005,post_2009,post_2013,post_2017,first_vote,second_vote,Wähler,...,Grüne,Andere,GEN,treated_0,treated_10,treated_20,treated_30,treated_50,treated_100,_merge
0,01001000,2002.0,SH,0.0,0.0,0.0,0.0,1.0,NaN,74.618193,...,5.835101,0.399580,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
1,01001000,2002.0,SH,0.0,0.0,0.0,0.0,NaN,1.0,74.618193,...,11.443165,1.968982,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
2,01001000,2005.0,SH,1.0,0.0,0.0,0.0,1.0,NaN,73.170076,...,5.656662,0.802172,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
3,01001000,2005.0,SH,1.0,0.0,0.0,0.0,NaN,1.0,73.170076,...,10.423966,2.145745,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
4,01001000,2009.0,SH,1.0,1.0,0.0,0.0,1.0,NaN,67.438292,...,12.164217,1.097424,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
5,01001000,2009.0,SH,1.0,1.0,0.0,0.0,NaN,1.0,67.889787,...,15.561352,4.911207,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
6,01001000,2013.0,SH,1.0,1.0,1.0,0.0,1.0,NaN,66.590350,...,9.803625,6.674164,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
7,01001000,2013.0,SH,1.0,1.0,1.0,0.0,NaN,1.0,66.590350,...,11.903167,9.296444,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
8,01001000,2017.0,SH,1.0,1.0,1.0,1.0,1.0,NaN,70.885101,...,12.107541,8.532992,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both
9,01001000,2017.0,SH,1.0,1.0,1.0,1.0,NaN,1.0,70.885101,...,15.051450,11.282499,Flensburg,0.0,0.0,0.0,0.0,0.0,1.0,both


Inspect merge results

In [36]:
print(btw_treat['_merge'].describe())

count     119708
unique         3
top         both
freq      103388
Name: _merge, dtype: object


Clean data

In [37]:
btw_treat = btw_treat[btw_treat['_merge'] == 'both']
btw_treat = btw_treat.drop(columns=['_merge'])

Merge in control variables

In [38]:
controls = pd.read_csv(f'{path}/data/controls.csv', converters = {'AGS': str})
btw_final = pd.merge(btw_treat, controls, on='AGS', how='outer', indicator=True)
btw_final.head(10)

,AGS,year,Land,post_2005,post_2009,post_2013,post_2017,first_vote,second_vote,Wähler,...,avg_age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion_other,_merge
0,01001000,2002.0,SH,0.0,0.0,0.0,0.0,1.0,NaN,74.618193,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
1,01001000,2002.0,SH,0.0,0.0,0.0,0.0,NaN,1.0,74.618193,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
2,01001000,2005.0,SH,1.0,0.0,0.0,0.0,1.0,NaN,73.170076,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
3,01001000,2005.0,SH,1.0,0.0,0.0,0.0,NaN,1.0,73.170076,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
4,01001000,2009.0,SH,1.0,1.0,0.0,0.0,1.0,NaN,67.438292,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
5,01001000,2009.0,SH,1.0,1.0,0.0,0.0,NaN,1.0,67.889787,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
6,01001000,2013.0,SH,1.0,1.0,1.0,0.0,1.0,NaN,66.590350,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
7,01001000,2013.0,SH,1.0,1.0,1.0,0.0,NaN,1.0,66.590350,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
8,01001000,2017.0,SH,1.0,1.0,1.0,1.0,1.0,NaN,70.885101,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both
9,01001000,2017.0,SH,1.0,1.0,1.0,1.0,NaN,1.0,70.885101,...,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726,both


In [39]:
print(btw_final['_merge'].describe())

count     106156
unique         3
top         both
freq      103326
Name: _merge, dtype: object


In [40]:
btw_final = btw_final[btw_final['_merge'] == 'both']
btw_final = btw_final.set_index(['AGS', 'GEN', 'year'])
btw_final = btw_final.drop(columns=['_merge'])

Drop obs. with missing values for election results

In [41]:
btw_final = btw_final.dropna(axis='rows', how='any', subset=['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'])
btw_final.shape

(103320, 36)

Export dataset

In [42]:
btw_final.to_csv(f'{path}/data/btw.csv', encoding = 'utf-8-sig')
btw_final.head()

Land  post_2005  post_2009  post_2013  post_2017  \
AGS      GEN       year                                                      
01001000 Flensburg 2002.0   SH        0.0        0.0        0.0        0.0   
                   2002.0   SH        0.0        0.0        0.0        0.0   
                   2005.0   SH        1.0        0.0        0.0        0.0   
                   2005.0   SH        1.0        0.0        0.0        0.0   
                   2009.0   SH        1.0        1.0        0.0        0.0   

                           first_vote  second_vote     Wähler      Union  \
AGS      GEN       year                                                    
01001000 Flensburg 2002.0         1.0          NaN  74.618193  33.815987   
                   2002.0         NaN          1.0  74.618193  29.580047   
                   2005.0         1.0          NaN  73.170076  35.142194   
                   2005.0         NaN          1.0  73.170076  28.837981   
                   2009.0         1.0          NaN  67.438292  30.540558   

                                 SPD  ...  avg_income  avg_age  age 0-17  \
AGS      GEN       year               ...                                  
01001000 Flensburg 2002.0  54.188379  ...     27000.0     43.0      15.2   
                   2002.0  48.186518  ...     27000.0     43.0      15.2   
                   2005.0  50.288119  ...     27000.0     43.0      15.2   
                   2005.0  43.536851  ...     27000.0     43.0      15.2   
                   2009.0  36.475021  ...     27000.0     43.0      15.2   

                           age 18-24  age 25-44  age 45-64  age 65+  \
AGS      GEN       year                                               
01001000 Flensburg 2002.0       11.3       26.3       26.6     20.6   
                   2002.0       11.3       26.3       26.6     20.6   
                   2005.0       11.3       26.3       26.6     20.6   
                   2005.0       11.3       26.3       26.6     20.6   
                   2009.0       11.3       26.3       26.6     20.6   

                           protestant  catholic  religion_other  
AGS      GEN       year                                          
01001000 Flensburg 2002.0    0.496815  0.063459        0.439726  
                   2002.0    0.496815  0.063459        0.439726  
                   2005.0    0.496815  0.063459        0.439726  
                   2005.0    0.496815  0.063459        0.439726  
                   2009.0    0.496815  0.063459        0.439726  

[5 rows x 36 columns]